In [1]:
%run ./MovieLens_Ranking.ipynb

In [2]:
# 相邻算法
def PCC_item(k, j):
    # 求物品k和j的交集
    u_k = U_j(k)
    u_j = U_j(j)
    intersection = np.intersect1d(u_k, u_j)
    union = np.union1d(u_k, u_j)
    if len(union) == 0:
        return 0
    return len(intersection) / len(union)

def PCC_user(u, w):
    # 求用户u和w的交集
    i_u = I_u(u)
    i_w = I_u(w)
    intersection = np.intersect1d(i_u, i_w)
    union = np.union1d(i_u, i_w)
    if len(union) == 0:
        return 0
    return len(intersection) / len(union)

In [3]:
# 求用户和物品的相邻系数矩阵
s_uw = np.zeros((user_num + 1, user_num + 1), float)
for i in range(1, user_num + 1):
    for j in range(1, i + 1):
        if j == i:
            s_uw[i][j] = 1.0
        else:
            s_uw[i][j] = PCC_user(i, j)
            s_uw[j][i] = PCC_user(i, j)
            
s_kj = np.zeros((item_num + 1, item_num + 1), float)
for i in range(1, item_num + 1):
    for j in range(1, i + 1):
        if j == i:
            s_kj[i][j] = 1.0
        else:
            s_kj[i][j] = PCC_item(i, j)
            s_kj[j][i] = PCC_item(i, j)

In [4]:
# 获取k个邻居
def get_user_neighbors(u, j, k=50):
    neighbors = np.where(s_uw[u] != 0)[0]
    neighbors = neighbors[neighbors != u]
    neighbors = np.intersect1d(neighbors, U_j(j))
    # 获取用户u与邻居的相关系数数组
    coefficients = s_uw[u][neighbors]
    # 使用argsort函数获取绝对值最大的k个邻居的索引
    return neighbors[np.argsort(coefficients)[::-1][:k]]

def get_item_neighbors(u, j, k=50):
    neighbors = np.where(s_kj[j] != 0)[0]
    neighbors = neighbors[neighbors != j]
    neighbors = np.intersect1d(neighbors, I_u(u))
    # 获取物品j与邻居的相关系数数组
    coefficients = s_kj[j][neighbors]
    # 使用argsort函数获取绝对值最大的k个邻居的索引
    return neighbors[np.argsort(coefficients)[::-1][:k]]

In [5]:
def I_OCCF(u, j):
    # 选取邻居
    neighbors = get_item_neighbors(u, j)
    rating = 0
    for k in neighbors:
        rating += s_kj[k][j]
    return rating

def U_OCCF(u, j):
    # 选取邻居
    neighbors = get_user_neighbors(u, j)
    rating = 0
    for w in neighbors:
        rating += s_uw[u][w]
    return rating

def H_OCCF(u, j):
    return I_OCCF(u, j) / 2 + U_OCCF(u, j) / 2

In [6]:
predict(I_OCCF, 5)

Pre: 0.3531
Rec: 0.1147


In [7]:
predict(U_OCCF, 5)

Pre: 0.3917
Rec: 0.1263


In [8]:
predict(H_OCCF, 5)

Pre: 0.3851
Rec: 0.1286
